In [50]:
!apt-get install -y xvfb # Install X Virtual Frame Buffer
import os
os.system('Xvfb :1 -screen 0 1600x1200x16  &')    # create virtual display with size 1600x1200 and 16 bit color. Color can be changed to 24 or 8
os.environ['DISPLAY']=':1.0'    # tell X clients to use our virtual DISPLAY :1.0.

Reading package lists... Done
Building dependency tree       
Reading state information... Done
xvfb is already the newest version (2:1.19.6-1ubuntu4.8).
0 upgraded, 0 newly installed, 0 to remove and 31 not upgraded.


# Import the necessary libraries

In [5]:
from pprint import pprint
import spacy
from spacy import displacy
from spacy.lang.en import English
from collections import Counter
import en_core_web_sm
import pandas as pd
import os
import regex as re
import nltk
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download("all")
nlp = en_core_web_sm.load()



[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Package cess_esp is already up-to-date!
[nltk_data]    | Downloading packag

# NER using Spacy

## Data Extraction

In [6]:
RootPath = r'/content/drive/MyDrive/ACU_OCR_TXT'
ProcessData , fileNames = [], []
for filename in os.listdir(RootPath):
    if filename.endswith(".txt") :
      dataPath = os.path.join(RootPath, filename)
      f = open(dataPath, "r")
      ProcessData.append(f.read())
      fileNames.append(filename)
    else:
      continue

In [7]:
inputData = pd.DataFrame({"FileName":fileNames, "Text":ProcessData}) # Creating Data Frame of all the text files

In [8]:
def decontracted(phrase):
  """
  This function modifies the text like won't -> will not
  """
  # specific
  phrase = phrase.replace("\n", " ")
  phrase = re.sub(r"won't", "will not", phrase)
  phrase = re.sub(r"can\'t", "can not", phrase)

  # general
  phrase = re.sub(r"n\'t", " not", phrase)
  phrase = re.sub(r"\'re", " are", phrase)
  phrase = re.sub(r"\'s", " is", phrase)
  phrase = re.sub(r"\'d", " would", phrase)
  phrase = re.sub(r"\'ll", " will", phrase)
  phrase = re.sub(r"\'t", " not", phrase)
  phrase = re.sub(r"\'ve", " have", phrase)
  phrase = re.sub(r"\'m", " am", phrase)
  return phrase

In [9]:
inputData['ContractedData'] = inputData['Text'].apply(decontracted) # Creating the modified texxt

## Tokenization using spaCY

In [11]:
# Word tokenization
def TokenizeText(text):
  """ 
  Tokenize the text intput into words
  """

  # Load English tokenizer, tagger, parser, NER and word vectors
  nlp = English()
  #  "nlp" Object is used to create documents with linguistic annotations.
  my_doc = nlp(text)
  # Create list of word tokens
  token_string = ''
  for token in my_doc:
      token_string += token.text + ' '
  #token_string = token_string.replace("\n", " ")
  return token_string


In [12]:
inputData['TokenisedData'] = inputData['ContractedData'].apply(TokenizeText)

In [15]:
def ApplyLemma(text):
  """
  Apply Lemmatization on the text, this is a kind of noarmalization technique on the text data 
  """
  nlp = spacy.load('en') # Load english language/ dictionary
  doc = nlp(text)
  finText = ""
  for token in doc:
      finText += " " + token.lemma_
  return finText

In [16]:
inputData['LemmetizedData'] = inputData['TokenisedData'].apply(ApplyLemma)

In [17]:
inputData.head(10)

,FileName,Text,ContractedData,TokenisedData,LemmetizedData
0,ACU0000042_ocr.txt,2\n3\n5\n11\n9 |.\n21\n14\n13\n12\nMETRIC 1\n5...,2 3 5 11 9 |. 21 14 13 12 METRIC 1 5 ABILENE C...,2 3 5 11 9 | . 21 14 13 12 METRIC 1 5 ABILENE ...,2 3 5 11 9 | . 21 14 13 12 METRIC 1 5 abilene...
1,ACU0000024_ocr.txt,2\n3\n12\n101\n6\n8\n14\n13\n11\nMETRIC 1\n2\n...,2 3 12 101 6 8 14 13 11 METRIC 1 2 ABILENE CHR...,2 3 12 101 6 8 14 13 11 METRIC 1 2 ABILENE CHR...,2 3 12 101 6 8 14 13 11 metric 1 2 abilene CH...
2,ACU0000006_ocr.txt,5\n2\n3.\n9\nABILENE CHRISTIAN COLLEGE\n6 | 8\...,5 2 3. 9 ABILENE CHRISTIAN COLLEGE 6 | 8 METRI...,5 2 3 . 9 ABILENE CHRISTIAN COLLEGE 6 | 8 METR...,5 2 3 . 9 abilene CHRISTIAN college 6 | 8 met...
3,ACU0000023_ocr.txt,2\n3\n5\n14\n12\n11\n13\nMETRIC 1\n3.\n2\nABIL...,2 3 5 14 12 11 13 METRIC 1 3. 2 ABILENE CHRIST...,2 3 5 14 12 11 13 METRIC 1 3 . 2 ABILENE CHRIS...,2 3 5 14 12 11 13 METRIC 1 3 . 2 abilene CHRI...
4,ACU0000039_ocr.txt,2\n3.\n5\n6\n13\n6 |\n8\n31\n21\n14\nMETRIC 1\...,2 3. 5 6 13 6 | 8 31 21 14 METRIC 1 12 ABILENE...,2 3 . 5 6 13 6 | 8 31 21 14 METRIC 1 12 ABILEN...,2 3 . 5 6 13 6 | 8 31 21 14 METRIC 1 12 ABILE...
5,ACU0000030_ocr.txt,2\n3.\n13\n11\n6\n8\n9.\n14\nMETRIC 1\n12\n2\n...,2 3. 13 11 6 8 9. 14 METRIC 1 12 2 ABILENE CHR...,2 3 . 13 11 6 8 9 . 14 METRIC 1 12 2 ABILENE C...,2 3 . 13 11 6 8 9 . 14 metric 1 12 2 abilene ...
6,ACU0000079_ocr.txt,ES\n2\n3\n6.\nABILENE CHRISTIAN COLLEGE\n14\nH...,ES 2 3 6. ABILENE CHRISTIAN COLLEGE 14 HERBARI...,ES 2 3 6 . ABILENE CHRISTIAN COLLEGE 14 HERBAR...,ES 2 3 6 . abilene CHRISTIAN COLLEGE 14 herba...
7,ACU0000019_ocr.txt,2\n3.\n6.\n13\n6 |\n8\n21\n14\nMETRIC 1\n12\n5...,2 3. 6. 13 6 | 8 21 14 METRIC 1 12 5 ABILENE C...,2 3 . 6 . 13 6 | 8 21 14 METRIC 1 12 5 ABILENE...,2 3 . 6 . 13 6 | 8 21 14 METRIC 1 12 5 abilen...
8,ACU0000007_ocr.txt,5\n6\n2\n3\n9 |\nMETRIC 1\n101\n6 |\n8\n2\n11\...,5 6 2 3 9 | METRIC 1 101 6 | 8 2 11 14 12 ABIL...,5 6 2 3 9 | METRIC 1 101 6 | 8 2 11 14 12 ABIL...,5 6 2 3 9 | METRIC 1 101 6 | 8 2 11 14 12 ABI...
9,ACU0000086_ocr.txt,2\n3.\n11\n31\n14\n13\n12\nMETRIC 1\n8\n2\nIMA...,2 3. 11 31 14 13 12 METRIC 1 8 2 IMAGED 20 NOV...,2 3 . 11 31 14 13 12 METRIC 1 8 2 IMAGED 20 NO...,2 3 . 11 31 14 13 12 METRIC 1 8 2 imaged 20 N...


In [23]:
filter_sent = lambda x : ' '.join( re.findall('\w+',x )) # filter only alpha numeric values from the text
inputData['FinalData'] = inputData['LemmetizedData'].apply(filter_sent)

In [24]:
inputData.head(10)

,FileName,Text,ContractedData,TokenisedData,LemmetizedData,test,FinalData
0,ACU0000042_ocr.txt,2\n3\n5\n11\n9 |.\n21\n14\n13\n12\nMETRIC 1\n5...,2 3 5 11 9 |. 21 14 13 12 METRIC 1 5 ABILENE C...,2 3 5 11 9 | . 21 14 13 12 METRIC 1 5 ABILENE ...,2 3 5 11 9 | . 21 14 13 12 METRIC 1 5 abilene...,2 3 5 11 9 21 14 13 12 METRIC 1 5 abilene CHRI...,2 3 5 11 9 21 14 13 12 METRIC 1 5 abilene CHRI...
1,ACU0000024_ocr.txt,2\n3\n12\n101\n6\n8\n14\n13\n11\nMETRIC 1\n2\n...,2 3 12 101 6 8 14 13 11 METRIC 1 2 ABILENE CHR...,2 3 12 101 6 8 14 13 11 METRIC 1 2 ABILENE CHR...,2 3 12 101 6 8 14 13 11 metric 1 2 abilene CH...,2 3 12 101 6 8 14 13 11 metric 1 2 abilene CHR...,2 3 12 101 6 8 14 13 11 metric 1 2 abilene CHR...
2,ACU0000006_ocr.txt,5\n2\n3.\n9\nABILENE CHRISTIAN COLLEGE\n6 | 8\...,5 2 3. 9 ABILENE CHRISTIAN COLLEGE 6 | 8 METRI...,5 2 3 . 9 ABILENE CHRISTIAN COLLEGE 6 | 8 METR...,5 2 3 . 9 abilene CHRISTIAN college 6 | 8 met...,5 2 3 9 abilene CHRISTIAN college 6 8 metric 1...,5 2 3 9 abilene CHRISTIAN college 6 8 metric 1...
3,ACU0000023_ocr.txt,2\n3\n5\n14\n12\n11\n13\nMETRIC 1\n3.\n2\nABIL...,2 3 5 14 12 11 13 METRIC 1 3. 2 ABILENE CHRIST...,2 3 5 14 12 11 13 METRIC 1 3 . 2 ABILENE CHRIS...,2 3 5 14 12 11 13 METRIC 1 3 . 2 abilene CHRI...,2 3 5 14 12 11 13 METRIC 1 3 2 abilene CHRISTI...,2 3 5 14 12 11 13 METRIC 1 3 2 abilene CHRISTI...
4,ACU0000039_ocr.txt,2\n3.\n5\n6\n13\n6 |\n8\n31\n21\n14\nMETRIC 1\...,2 3. 5 6 13 6 | 8 31 21 14 METRIC 1 12 ABILENE...,2 3 . 5 6 13 6 | 8 31 21 14 METRIC 1 12 ABILEN...,2 3 . 5 6 13 6 | 8 31 21 14 METRIC 1 12 ABILE...,2 3 5 6 13 6 8 31 21 14 METRIC 1 12 ABILENE CH...,2 3 5 6 13 6 8 31 21 14 METRIC 1 12 ABILENE CH...
5,ACU0000030_ocr.txt,2\n3.\n13\n11\n6\n8\n9.\n14\nMETRIC 1\n12\n2\n...,2 3. 13 11 6 8 9. 14 METRIC 1 12 2 ABILENE CHR...,2 3 . 13 11 6 8 9 . 14 METRIC 1 12 2 ABILENE C...,2 3 . 13 11 6 8 9 . 14 metric 1 12 2 abilene ...,2 3 13 11 6 8 9 14 metric 1 12 2 abilene CHRIS...,2 3 13 11 6 8 9 14 metric 1 12 2 abilene CHRIS...
6,ACU0000079_ocr.txt,ES\n2\n3\n6.\nABILENE CHRISTIAN COLLEGE\n14\nH...,ES 2 3 6. ABILENE CHRISTIAN COLLEGE 14 HERBARI...,ES 2 3 6 . ABILENE CHRISTIAN COLLEGE 14 HERBAR...,ES 2 3 6 . abilene CHRISTIAN COLLEGE 14 herba...,ES 2 3 6 abilene CHRISTIAN COLLEGE 14 herbariu...,ES 2 3 6 abilene CHRISTIAN COLLEGE 14 herbariu...
7,ACU0000019_ocr.txt,2\n3.\n6.\n13\n6 |\n8\n21\n14\nMETRIC 1\n12\n5...,2 3. 6. 13 6 | 8 21 14 METRIC 1 12 5 ABILENE C...,2 3 . 6 . 13 6 | 8 21 14 METRIC 1 12 5 ABILENE...,2 3 . 6 . 13 6 | 8 21 14 METRIC 1 12 5 abilen...,2 3 6 13 6 8 21 14 METRIC 1 12 5 abilene CHRIS...,2 3 6 13 6 8 21 14 METRIC 1 12 5 abilene CHRIS...
8,ACU0000007_ocr.txt,5\n6\n2\n3\n9 |\nMETRIC 1\n101\n6 |\n8\n2\n11\...,5 6 2 3 9 | METRIC 1 101 6 | 8 2 11 14 12 ABIL...,5 6 2 3 9 | METRIC 1 101 6 | 8 2 11 14 12 ABIL...,5 6 2 3 9 | METRIC 1 101 6 | 8 2 11 14 12 ABI...,5 6 2 3 9 METRIC 1 101 6 8 2 11 14 12 ABILENE ...,5 6 2 3 9 METRIC 1 101 6 8 2 11 14 12 ABILENE ...
9,ACU0000086_ocr.txt,2\n3.\n11\n31\n14\n13\n12\nMETRIC 1\n8\n2\nIMA...,2 3. 11 31 14 13 12 METRIC 1 8 2 IMAGED 20 NOV...,2 3 . 11 31 14 13 12 METRIC 1 8 2 IMAGED 20 NO...,2 3 . 11 31 14 13 12 METRIC 1 8 2 imaged 20 N...,2 3 11 31 14 13 12 METRIC 1 8 2 imaged 20 NOV ...,2 3 11 31 14 13 12 METRIC 1 8 2 imaged 20 NOV ...


In [25]:

def getNER(text):
  """
  Creating the Named Entity Relation for the text given as input using Spacy
  """
  nlp = en_core_web_sm.load()
  sentence_nlp = nlp(text)
  # print named entities in article
  NER_List = [(word, word.ent_type_) for word in sentence_nlp if word.ent_type_]
  # visualize named entities
  #displacy.render(sentence_nlp, style='ent', jupyter=True)
  return NER_List

In [26]:
inputData['NER_Data'] = inputData['FinalData'].apply(getNER) # NER for each file is created in the column NER Data

In [27]:
inputData.set_index("FileName", inplace=True) #Setting index as the name of the files, which helps in easy acccess of the data for a particular file

In [28]:
#NER for a given sentence, displacy the NER text
nlp = en_core_web_sm.load()
# Index the data based on the text file name, then see the NER for the contents of the file by running the below code
sentenceNER = nlp(inputData.loc['ACU0000001_ocr.txt', 'test'])
displacy.render(sentenceNER, style='ent', jupyter=True)

## NER for the complete data(100 text files)

In [29]:
named_entities = []
for sentence in inputData['test']:
  sentence = sentence.replace("\n", ' ' )
  temp_entity_name = ''
  temp_named_entity = None
  sentence = nlp(sentence)
  for word in sentence:
      term = word.text 
      tag = word.ent_type_
      if tag:
          temp_entity_name = ' '.join([temp_entity_name, term]).strip()
          temp_named_entity = (temp_entity_name, tag)
      else:
          if temp_named_entity:
              named_entities.append(temp_named_entity)
              temp_entity_name = ''
              temp_named_entity = None
# Overall NER for all the text files
entity_frame = pd.DataFrame(named_entities, 
                        columns=['Entity Name', 'Entity Type'])

In [30]:
# get the top named entities
top_entities = (entity_frame.groupby(by=['Entity Name', 'Entity Type'])
                           .size()
                           .sort_values(ascending=False)
                           .reset_index().rename(columns={0 : 'Frequency'}))
top_entities.T.iloc[:,:50]


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
Entity Name,20,2,TM,2 3,Abilene Christian University,CHRISTIAN UNIVERSITY,2020,6,CHRISTIAN COLLEGE,14,CHRISTIAN,12,3,2 3 5,Habitat,1,5,HABITAT,14 metric,Texas,Barbara Bowles,ABILENE CHRISTIAN,Christian University,Altitude Habitat,8,Soil,ABILENE CHRISTIAN COLLEGE,Pinaceae,Abilene Christian,HERBARIUM,14 6,2020,4,ACC,24colorcard tm,CHRISTIAN COLLEGE HERBARIUM,Herbarium,IMAGED,12 6,101,canadian,1 12,STATE,15,Jason Peters Abilene Christian University,Robert Patton,3 14 metric,83,MM CM,10
Entity Type,CARDINAL,CARDINAL,ORG,CARDINAL,ORG,ORG,DATE,CARDINAL,ORG,CARDINAL,ORG,CARDINAL,CARDINAL,CARDINAL,ORG,CARDINAL,CARDINAL,ORG,QUANTITY,GPE,PERSON,ORG,ORG,PERSON,CARDINAL,ORG,ORG,NORP,NORP,PRODUCT,CARDINAL,CARDINAL,CARDINAL,ORG,PERSON,ORG,PRODUCT,ORG,CARDINAL,CARDINAL,NORP,CARDINAL,ORG,CARDINAL,ORG,PERSON,QUANTITY,CARDINAL,PERSON,CARDINAL
Frequency,78,49,46,30,26,22,18,16,16,12,11,11,10,9,9,7,7,6,6,6,6,6,6,5,5,4,4,4,4,3,3,3,3,3,3,3,3,3,3,3,3,2,2,2,2,2,2,2,2,2


In [32]:
# get the top(15) named entity types
top_entities = (entity_frame.groupby(by=['Entity Type'])
                           .size()
                           .sort_values(ascending=False)
                           .reset_index().rename(columns={0 : 'Frequency'}))
top_entities.T.iloc[:,:15]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
Entity Type,CARDINAL,ORG,PERSON,DATE,PRODUCT,NORP,GPE,QUANTITY,TIME,FAC,EVENT,LOC,PERCENT,LAW,WORK_OF_ART
Frequency,332,314,139,75,44,26,26,23,8,8,6,4,2,2,1


# NER using NLTK

In [33]:
NLTK_Data = pd.DataFrame({"FileName":fileNames, "Text":ProcessData}) # Data frame creation has data for all the text files

In [34]:
NLTK_Data['ContractedData'] = NLTK_Data['Text'].apply(decontracted) # Data preprocessing

In [35]:
def lemmatize(textList):
  """
  Data Normalisation technique, lemmatization for text data using NLTK
  """
  lemmatizer = WordNetLemmatizer()

  return [lemmatizer.lemmatize(word) for word in textList] 

In [37]:
# TOkenization using NLTK
NLTK_Data['ToknData'] = NLTK_Data['ContractedData'].apply(lambda x : nltk.word_tokenize(x))
# Stemming using NLTK
NLTK_Data['Lemma_Data'] = NLTK_Data['ToknData'].apply(lemmatize)


In [40]:
def GenNER_NLTK(sentence):
  """
  Create NER for a given text file using NLTK
  """
  word_tree = nltk.ne_chunk(nltk.tag.pos_tag(sentence)) # Creating NER for the text
  chunked = [] 
  # Extracting the leaves from the NER trees
  for t in word_tree.subtrees():
        chunked.append(t)
  named_entities = chunked[1:]      
  return named_entities
 


In [41]:
NLTK_Data['NER_data'] = NLTK_Data['Lemma_Data'].apply(GenNER_NLTK)

In [42]:
NLTK_Data.head(5)

,FileName,Text,ContractedData,ToknData,Lemma_Data,NER_data
0,ACU0000042_ocr.txt,2\n3\n5\n11\n9 |.\n21\n14\n13\n12\nMETRIC 1\n5...,2 3 5 11 9 |. 21 14 13 12 METRIC 1 5 ABILENE C...,"[2, 3, 5, 11, 9, |, ., 21, 14, 13, 12, METRIC,...","[2, 3, 5, 11, 9, |, ., 21, 14, 13, 12, METRIC,...","[[(METRIC, NNP)], [(ABILENE, NNP)], [(CHRISTIA..."
1,ACU0000024_ocr.txt,2\n3\n12\n101\n6\n8\n14\n13\n11\nMETRIC 1\n2\n...,2 3 12 101 6 8 14 13 11 METRIC 1 2 ABILENE CHR...,"[2, 3, 12, 101, 6, 8, 14, 13, 11, METRIC, 1, 2...","[2, 3, 12, 101, 6, 8, 14, 13, 11, METRIC, 1, 2...","[[(METRIC, NNP)], [(ABILENE, NNP)], [(Common, ..."
2,ACU0000006_ocr.txt,5\n2\n3.\n9\nABILENE CHRISTIAN COLLEGE\n6 | 8\...,5 2 3. 9 ABILENE CHRISTIAN COLLEGE 6 | 8 METRI...,"[5, 2, 3, ., 9, ABILENE, CHRISTIAN, COLLEGE, 6...","[5, 2, 3, ., 9, ABILENE, CHRISTIAN, COLLEGE, 6...","[[(ABILENE, NNP)], [(METRIC, NNP)], [(HERBARIU..."
3,ACU0000023_ocr.txt,2\n3\n5\n14\n12\n11\n13\nMETRIC 1\n3.\n2\nABIL...,2 3 5 14 12 11 13 METRIC 1 3. 2 ABILENE CHRIST...,"[2, 3, 5, 14, 12, 11, 13, METRIC, 1, 3, ., 2, ...","[2, 3, 5, 14, 12, 11, 13, METRIC, 1, 3, ., 2, ...","[[(METRIC, NNP)], [(ABILENE, NNP)], [(CHRISTIA..."
4,ACU0000039_ocr.txt,2\n3.\n5\n6\n13\n6 |\n8\n31\n21\n14\nMETRIC 1\...,2 3. 5 6 13 6 | 8 31 21 14 METRIC 1 12 ABILENE...,"[2, 3, ., 5, 6, 13, 6, |, 8, 31, 21, 14, METRI...","[2, 3, ., 5, 6, 13, 6, |, 8, 31, 21, 14, METRI...","[[(METRIC, NNP)], [(ABILENE, NNP)], [(NO, NNP)..."


In [46]:
NLTK_Data.set_index("FileName",inplace=True) # Setting the File Name column to index which facilitates in easy indexing

In [45]:
samplText = NLTK_Data['NER_data']['ACU0000042_ocr.txt']

In [48]:
pattern = 'NP: {<DT>?<JJ>*<NN>}'
cp = nltk.RegexpParser(pattern)
cs = cp.parse(samplText)


In [52]:
pprint(cs)

Tree('S', [Tree('ORGANIZATION', [('METRIC', 'NNP')]), Tree('ORGANIZATION', [('ABILENE', 'NNP')]), Tree('ORGANIZATION', [('CHRISTIAN', 'NNP')]), Tree('ORGANIZATION', [('UNIVERSITY', 'NNP')]), Tree('ORGANIZATION', [('HERBARIUM', 'NNP')]), Tree('ORGANIZATION', [('RIRIRIRIRIA', 'NNP'), ('Herbarium', 'NNP')]), Tree('GPE', [('No', 'NNP')]), Tree('PERSON', [('Family', 'NNP')]), Tree('ORGANIZATION', [('Locality', 'NNP'), ('Colmawcho', 'NNP'), ('Count', 'NNP')]), Tree('ORGANIZATION', [('Date', 'NNP'), ('Altitude', 'NNP'), ('Habitat', 'NNP')]), Tree('PERSON', [('Rhowda', 'NNP')]), Tree('PERSON', [('Abilene', 'NNP'), ('Christian', 'NNP'), ('University', 'NNP')]), Tree('ORGANIZATION', [('RIRIRIRIIRIRIRIRIIRIRIIRIRIRIIRIRIIRIRIII', 'NNP')])])


In [ ]:
cs.draw()